# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# A for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/jameskhamthung/Desktop/sparkify_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#total number of rows 
#print(len(full_data_rows_list))
#list of event data rows
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26
3,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61
4,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80


In [6]:
list(df.columns) 

['artist',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'sessionId',
 'song',
 'userId']

# Part II. Apache Cassandra coding. 

#### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1']) # This should make a connection to a Cassandra instance your local machine 
session = cluster.connect() # To establish connection and begin executing queries, need a session

#### Create Keyspace

In [8]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [9]:
session.set_keyspace('sparkify')

### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession  = 4

#### Create Table 1

In [10]:
query1 = """CREATE TABLE IF NOT EXISTS songplay_session
            (session_id int, 
            session_item_id int,
            artist_name text, 
            song_title text, 
            song_length float,  
            PRIMARY KEY (session_id, session_item_id))"""

session.execute(query1)                  

#### Insert into Table 1

In [13]:
with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = """INSERT INTO songplay_session
                    (session_id, 
                    session_item_id,
                    artist_name, 
                    song_title, 
                    song_length)
                    VALUES (%s, %s, %s, %s, %s)"""
        insert_values = (int(line[8]), int(line[3]),line[0], line[9], float(line[5]))
        
        session.execute(query1, insert_values)

#### Select Statement for Table 1

In [14]:
query1 = "select * FROM songplay_session WHERE session_id=338 AND session_item_id=4"
rows = session.execute(query1)
df1 = pd.DataFrame(rows, columns=['session','item','artist','song','length'])
df1
#for row in rows: print (row.artist_name, row.song_title,row.song_length, row.session_id, row.session_item_id)

,session,item,artist,song,length
0,338,4,Faithless,495.307312,Music Matters (Mark Knight Dub)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create Table 2

In [15]:
query2 = """CREATE TABLE IF NOT EXISTS songplay_user_session 
            (artist_name text, 
            song_title text,
            user_id int, 
            first_name text, 
            last_name text,
            session_id int, 
            session_item_id int,
            PRIMARY KEY ((user_id, session_id), session_item_id))"""

session.execute(query2) 

#### Insert into Table 2

In [16]:
with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query2 = """INSERT INTO songplay_user_session 
                    (user_id,
                    session_id, 
                    session_item_id,
                    artist_name, 
                    song_title,
                    first_name, 
                    last_name) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        insert_values =(int(line[10]),int(line[8]),int(line[3]),line[0], line[9],line[1], line[4])
        
        session.execute(query2, insert_values)

#### Select Statement for Table 2

In [17]:
query2 = "select * FROM songplay_user_session WHERE user_id=10 AND session_id=182"
rows = session.execute(query2)
df2 = pd.DataFrame(rows, columns=['user_id','session','item','artist','song','first name', 'last name'])
df2
#for row in rows:print (row.artist_name, row.song_title,row.user_id, row.first_name, row.last_name, row.session_id, row.session_item_id)

,user_id,session,item,artist,song,first name,last name
0,10,182,0,Down To The Bone,Sylvie,Cruz,Keep On Keepin' On
1,10,182,1,Three Drives,Sylvie,Cruz,Greece 2000
2,10,182,2,Sebastien Tellier,Sylvie,Cruz,Kilometer
3,10,182,3,Lonnie Gordon,Sylvie,Cruz,Catch You Baby (Steve Pitron & Max Sanna Radio...


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own

#### Create Table 3

In [18]:
query3 = """CREATE TABLE IF NOT EXISTS songplay_user
            (song_title text,
            user_id int,
            first_name text, 
            last_name text,
            PRIMARY KEY (song_title, user_id))"""

session.execute(query3) 
        

#### Insert into Table 3

In [19]:
with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query3 = """INSERT INTO songplay_user 
                    (song_title,
                    user_id,
                    first_name, 
                    last_name) 
                    VALUES (%s,%s,%s,%s)"""
        insert_values =(line[9],int(line[10]),line[1], line[4])
        
        session.execute(query3, insert_values)

#### Select Statement for Table 3

In [20]:
query3 = "select * FROM songplay_user WHERE song_title ='All Hands Against His Own'"
rows = session.execute(query3)
df3 = pd.DataFrame(rows, columns=['Song','User_id','First Name','Last Name'])
df3
#for row in rows:print (row.first_name, row.last_name, row.song_title)

,Song,User_id,First Name,Last Name
0,All Hands Against His Own,29,Jacqueline,Lynch
1,All Hands Against His Own,80,Tegan,Levine
2,All Hands Against His Own,95,Sara,Johnson


### Drop the tables before closing out the sessions

In [21]:
query1 = "drop table songplay_session"
query2 = "drop table songplay_user_session"
query3 = "drop table songplay_user"
rows = session.execute(query1)
rows = session.execute(query2)
rows = session.execute(query3)

### Close the session and cluster connection

In [22]:
session.shutdown()
cluster.shutdown()